In [1]:
%matplotlib inline

# Assignment 3

**DUE: Tuesday, February 14th 2022 at 11:59pm**

Turn in the assignment via Canvas.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).
#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

### Resources

[1] https://community.plotly.com/t/two-3d-surface-with-different-color-map/18931/2 (Different color plots plotly)

[2] https://plotly.com/python/3d-surface-plots/ [plotting surfaces]

[3] https://community.plotly.com/t/3d-scatter-plot-with-surface-plot/27556/5 [plotting scatter plots]

[4] https://community.plotly.com/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079 [color scales]

In [2]:
%matplotlib inline

In [3]:
NAME = "Omkar Ghanekar"
STUDENT_ID = "1926974"

## Problem 1: Local Search on the Ackley Surface
---

As discussed in class with Local Search problems the path to the goal is not as important as the goal itself. In this question we work with the Ackley function, a non-convex function typically used as a test for optimization algorithms.

The Achkey function has many local optima but only one global optimum that is $f(0,0) = 0$. Interestingly enough it might not be that easy to find this solution.

The Ackley function is defined as:

$$f(x,y):= -20exp[-0.2\sqrt{0.5(x^2 + y^2)}] -exp[0.5(cos(2\pi x) + cos(2\pi y))] + e + 20$$

The figure below illustrates the result of running three optimization methods on the Ackley Function. These solutions were found using methods A, B and C.


![](https://drive.google.com/uc?id=1K0_iokZu-TgjD0xrfGmWQbb-r67kINI0)


In this question, you will implement 3 optimization algorithms: 

1.   Stochastic Hill Climbing with Restarts (SHCR)
2.   Simulated Annealing (SA)
3.   Local Beam Search (LBS)

**Deliverables**

1. Complete this Notebook with implementation of SHCR, SA and LBS.
2. For each algorithm **visualize** the candidate solutions found at each
step using the provided ```create_ackley_figure()``` function. please see the image above an example of an image that should be included in your submission.



In [4]:
import numpy as np
from numpy import arange
from numpy import exp
from numpy import sqrt
from numpy import cos
from numpy import e
from numpy import pi
from numpy import meshgrid
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
# hill climbing search of the ackley objective function
from numpy import asarray
import plotly.graph_objects as go

# objective function
def f(x, y):
	return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

def create_ackley_figure(solution_points=np.array([]), soln_names=['A','B,','C']):
  """
  :param solution_points A numpy array of dimensions [i, n, 2]
  where i is the number of solution points you want to plot and n is the 
  number of steps used in estimating your solution. In each case use the same 
  number of steps if you want to plot all solutions on the same figure.

  :return An interactive Ackley function figure with/without solution points.
  """
  # define range for input
  r_min, r_max = -5.0, 5.0
  # sample input range uniformly at 0.1 increments
  xaxis = arange(r_min, r_max, 0.1) # (100,)
  yaxis = arange(r_min, r_max, 0.1) # (100,)
  # create a mesh from the axis
  x, y = meshgrid(xaxis, yaxis)

  # compute targets
  results = f(x, y)

  figure = go.Figure(data=[go.Surface(z=results, x=x, y=y,colorscale='Blues',showscale=False)])
  figure.update_layout(title='Ackley Function', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  
  if solution_points.size:
    soln_colors = ['Reds', 'Rainbow', 'Viridis']
    for i in range(len(solution_points)):
      x_sln = solution_points[i][:,0]
      y_sln = solution_points[i][:,1]
      zdata = f(x_sln, y_sln)
      figure.add_scatter3d(name=soln_names[i], x=x_sln, y=y_sln, z=zdata, mode='markers',
      marker=dict(size=1, color=x_sln.flatten(), colorscale=soln_colors[i]))
      
      figure.update_layout(showlegend=True)
      
  return figure

In [5]:
### RUN THIS ###
"""
This is the Ackley Function.
"""
ack_figure = create_ackley_figure()
ack_figure

### General Function Definitions

In [6]:
# objective function
def objective(v):
  """
  This function defines the Ackley surface, it has a minimum of 0. This 
  can be used to test if we have found points that yield the minimum value.
  :param v, a tuple representing a 2D point being tested.

  returns a value in the range [-5.0, 5.0]
  """
  x, y = v
  return -20.0 * exp(-0.2 * sqrt(0.5 * (x**2 + y**2))) - exp(0.5 * (cos(2 * pi * x) + cos(2 * pi * y))) + e + 20

  
# check if a point is within the bounds of the search
def in_bounds(point, bounds=asarray([[-5.0, 5.0], [-5.0, 5.0]])):
  """
  It is possible our optimzation method could move us outside of our search 
  space, so it is helpful to check.
  :param point a tuple representing a 2D point
  :param a 2D array, that describes the bounds of each dimension of our point.
  returns a Boolean of whether the point lies in the bounds or not.
  """
	# enumerate all dimensions of the point
  for d in range(len(bounds)):
    # check if out of bounds for this dimension
    if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
      return False
  return True

### Part 1) Stochastic Hill Climbing with Restarts

1.   Implement SHCR in the cell below.
2.   Visualize and comment on the candidate points visited.


In [22]:
import random
# stochastic hill climbing with restarts algorithm
def stochastic_hillclimbing(objective, bounds, n_iterations, step_size, start_pt):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param start_pt the point we start optimizing from.
  returns [solution, solution_value, candidates]
  """
  ### YOUR CODE HERE ###
  
  solution, solution_value = start_pt, objective(start_pt)
  candidates = []
  for i in range(n_iterations):         
      # take a step and evaluate candidate point and check if we should keep the new point
      candidate = start_pt + randn(len(bounds)) * step_size 
      candidates.append(candidate)  
      candidate_eval = objective(candidate)
      if candidate_eval <= solution_value:
	  	    solution, solution_value = candidate, candidate_eval
			    # store the new point and print # print('>%d f(%s) = %.5f' % (i, solution, solution_value))
  return [solution, solution_value, candidates]

def random_restarts(objective, bounds, n_iter, step_size, n_restarts):
  """
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iter number of times to repeat stochastic hill climbing 
  :param step_size how much we should move
  :param n_restarts, the number of times we restart.
  returns [best solution, best solution value, visited points]
  """
  ### YOUR CODE HERE ###

  #state = problem.initial()
  count = 0
  best_sol, best_sol_val, best_sol_points = -99999, 99999 , []
  
  for iter in range(n_restarts):
      x = random.randrange(-5.0, 5.0)
      y = random.randrange(-5.0, 5.0)
      sol, sol_val, sol_points  = stochastic_hillclimbing(objective, bounds, n_iter, step_size,(x,y))
      if sol_val < best_sol_val:
          best_sol, best_sol_val, best_sol_points = sol, sol_val, sol_points
  return best_sol, best_sol_val, best_sol_points
  #return -1, -1, -1

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iter = 1000
# define the maximum step size
step_size = 0.01
# total number of random restarts
n_restarts = 30
# perform the hill climbing search
best, score, shcr_candidates = random_restarts(objective, bounds, n_iter, step_size, n_restarts)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f((0, 0)) = 0.000000


In [23]:
# An example of plotting solutions from 3 optimziation methods.
# all_method_candidates has shape [3, 1000, 2], the first dim is the number of solutions
# the second dim is the number of steps taken in each method.
# shcr_candidates = np.random.rand(100,2) # EXAMPLE PURPOSES ONLY replace with your actual solution.
#shcr_candidates = shcr_candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure

The restart random method helps the algorithm to reach the global minima at point(0.00, 0.00) with a cost of 0.0. It starts from 30 random points with a step size of 0.01 which help it reach very close to the global minima.

### Part 2) Simulated Annealing

1.   Implement SA in the cell below.
2.   Visualize and comment on the candidate points visited.

In [40]:
def get_temperature_schedule(epoch, temp, diff):
  """
  :param temp Temperature
  :param epoch Iterations elapsed
  :param diff Difference between value of candidate solution and value of best solution so far.
  """
  # calculate temperature for current epoch
  t = temp / float(epoch + 1)
  # calculate acceptance criterion
  criterion = exp(-diff / t)
  return criterion

# simulated annealing algorithm
def simulated_annealing(objective, bounds, n_iterations, step_size, temp):
  """ 
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param n_iterations number of times to repeat stochastic hill climbing
  :param step_size how much we should move
  :param temp temperature for each epoch
  returns [solution, solution_value, candidates] 
  """
  ### YOUR CODE HERE ###
  candidates = []
  best = (random.randrange(-5.0, 5.0), random.randrange(-5.0, 5.0))
  best_eval = objective(best)
  curr, curr_eval = best, best_eval
  for iter in range(n_iterations):
      candidate = curr + randn(len(bounds)) * step_size
      candidates.append(candidate)
        
      # take a step and evaluate candidate point check for new best solution
      candidate_eval = objective(candidate)
      if candidate_eval < best_eval:
          best, best_eval = candidate, candidate_eval
      diff = candidate_eval - curr_eval

      criterion = get_temperature_schedule(iter, temp, diff)
      if diff < 0 or rand() < criterion:
          curr, curr_eval = candidate, candidate_eval
  return [best, best_eval, candidates]

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 1000
# define the maximum step size
step_size = 0.1
# initial temperature
temp = 100
# perform the simulated annealing search
best, score, sa_candidates = simulated_annealing(objective, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))

Done!
f([ 0.00099391 -0.00211901]) = 0.006766


In [61]:
# sa_candidates = np.random.rand(40,2)
# sa_candidates = sa_candidates.reshape(-1,2)
all_method_candidates = np.array([shcr_candidates,sa_candidates])
ack_figure = create_ackley_figure(all_method_candidates)
ack_figure

The simulated annealing helps the algorithm to reach the global minima at point(-0.0001002   0.00013996) with a cost of 0.000488. It has a step size of 0.1 and runs for 1000 epochs, with the temprature of 100 which helps it reach very close to the global minima.

### Part 3) Local Beam Search



1.   Implement LBS in the cell below.
2.   Visualize and comment on the candidate points visited.
3.   Compare all 3 implementations by commenting on the distribution of points on the Ackley surface and the empirical run time of each method.


In [11]:
# generate the neighbors based on step size
def generate_neighbors(point, step_size=0.1):
    point = point.reshape(-1,1)
    possible_steps = possible_steps = np.array([[step_size*i for i in range(-10,11)],
                           [step_size*i for i in range(-10,11)]])
    neighbors = point + possible_steps
    return neighbors

def local_beam_search(objective, bounds, step_size, k, n_iterations):
  """ 
  :param objective, the function we are trying to optimize
  :param bounds, the boundaries of the problem
  :param k, how many candidates to consider
  :param n_iterations, how long to search for.
  returns [solution, solution_value, candidates] 
  """
  ### YOUR CODE HERE ###
  return -1, -1, -1

# seed the pseudorandom number generator
seed(240)
# define range for input
bounds = asarray([[-5.0, 5.0], [-5.0, 5.0]])
# define the total iterations
n_iterations = 10
# define the maximum step size
step_size = 0.1
# candidates to consider
k = 1
# perform the local beam search
sequences = local_beam_search(objective, bounds, step_size, k, n_iterations)

[YOUR ANSWER HERE]

## Problem 2: CSP
---

Consider the following constraint satisfaction problem. A linear graph has nodes of the following colors:

- Red
- Yellow
- Green
- Blue
- Violet

Each node has a domain of {1, 2, ..., 9}.

Each node type has the following constraints on its value:

- Red - No contraints
- Yellow - equals the rightmost digit of of the product of all its neighbors
- Green - equals the rightmost digit of the sum of all its neighbors 
- Blue - equals the leftmost digit of the sum of all its neighbors
- Violet - equals the leftmost digit of the product of all of its neighbors

As a reminder here is the pseudo code for the Min-Conflicts search algorithm:

![minconflicts](https://images2017.cnblogs.com/blog/1126979/201712/1126979-20171224140802287-1871895433.png)

**Notes:**

- It's possible that you won't converge to a solution in a single run. Try a few runs to see if you get to a solution.
- The testcases are to show you what a problem looks like, we will test/grade your program on different examples.

### Part 1) Implementation
Complete the function ```solve_csp``` defined below. You may find some helper functions useful.

In [12]:
import random
import copy

def create_arc(node_graph, source, dest, nodes):
    if nodes[source] in node_graph:
      node_graph[nodes[source]].append(nodes[dest])
    else:
      node_graph[nodes[source]] = [nodes[dest]] 

    return node_graph

def find_rightmost_digit(number):    # to find righmost number we do %10
    #print(number%10)
    return number%10

def find_leftmost_digit(number):     # to find leftmost number we find first char in str(number)
    #print(int(str(number)[0]))
    return int(str(number)[0])

def number_of_conflicts(node_graph, current, node_key_value):
    
    conflicts = []
    for node in node_graph:
      if not is_node_consistent(node_graph, node,current, node_key_value):
        conflicts.append(node)
    return conflicts


def is_node_consistent(node_graph, node, current, node_key_value):
    neighbours = node_graph[node]
    # print('in is consistent', node, neighbours)
    if node in ['Y','V']:
        neighbour_prod =1
        for neighbor in neighbours:
          neighbour_prod *= current[node_key_value[neighbor]]
        if node == 'Y' and find_rightmost_digit(neighbour_prod)== current[node_key_value[node]]:
            return True
        elif find_leftmost_digit(neighbour_prod) == current[node_key_value[node]]:
            return True
        return False
    elif node in ['G','B']:        
        neighbour_sum =0
        for neighbor in neighbours:
            neighbour_sum += current[node_key_value[neighbor]]
        if node == 'G' and find_rightmost_digit(neighbour_sum) == current[node_key_value[node]]:
            return True
        elif find_leftmost_digit(neighbour_sum) == current[node_key_value[node]]:
            return True
        return False
    return True


def assign_node_value(node, current, node_key_value, neighbours):
    # print('in number of conflicts', node, neighbours)
    if node in ['Y','V']:
        neighbour_prod =1
        for neighbor in neighbours:
          neighbour_prod *= current[node_key_value[neighbor]]
        #print(node, neighbour_prod, current, neighbours)
        if node == 'Y': 
            return find_rightmost_digit(neighbour_prod)
        else: 
            return find_leftmost_digit(neighbour_prod)
    elif node in ['G','B']:        
        neighbour_sum =0
        for neighbor in neighbours:
            neighbour_sum += current[node_key_value[neighbor]]
        #print(node, neighbour_sum, current, neighbours)
        if node == 'G':
            return find_rightmost_digit(neighbour_sum)
        else:
            return find_leftmost_digit(neighbour_sum)


def min_conflicts(node_graph, node_domains, max_steps, node_key_value):
    current = [1] *len(node_key_value) 
    #current = [random.randint(1,9) for i in range(len(node_key_value))]
    for i in range(max_steps):
        # number of conflicts in the current assignment of values
        conflicts = number_of_conflicts(node_graph, current, node_key_value)
        #print(i,len(conflicts),conflicts)
        # Check if we have a valid solution
        if not conflicts:
            return current

        conflict_cnt= {}

        # Tried with the minimum constraint node but it was stuck and never gave correct ans, hence tried generating some randomness, got solutions with that

        # make copy of current assignment to check how many conflicts new assignment might have
        # for node in conflicts:
        #     current_copy = copy.deepcopy(current)
        #     current_copy[node_key_value[node]] = assign_node_value(node, current_copy, node_key_value, node_graph[node])

        #     conflicts_after_assignment = number_of_conflicts(node_graph, current_copy, node_key_value)
        #     conflict_cnt[node] = len(conflicts_after_assignment)
        
        #print('conflicts after assignment',conflict_cnt)
  
        #scores = {key: value for key, value in sorted(conflict_cnt.items(), key=lambda item: item[1])}
        #current[node_key_value[next(iter(scores))]] = assign_node_value(node, current, node_key_value, node_graph[node])
        #print('sorted conflict cnt', scores, next(iter(scores)))

        # # Select a cell in conflict at random
        random_node =  random.choice(conflicts)
        current[node_key_value[random_node]] = assign_node_value(random_node, current, node_key_value, node_graph[random_node])


    return []

def solve_csp(nodes, arcs, max_steps):
    """
    This function solves the csp using the MinConflicts Search Algorithm.

    :param nodes, a list of letters that indicates what type of node it is,
                  the index of the node in the list indicates its id
                  letters = {R, Y, G, B, V}
                
    :param arcs,  a list of tuples that contains two numbers, indicating the 
                  IDs of the nodes the arc connects. 
                
    :param max_steps, max number of steps to make before giving up

    returns a list of values for the solution to the CSP where the 
             index of the value corresponds the value for that given node.
    """
    node_values = []
    ### YOUR CODE HERE ###

    # create a map of nodes and neighbours
    node_graph = {}
    for item in arcs:
      node_graph = create_arc(node_graph, item[0], item[1],nodes)
      node_graph = create_arc(node_graph, item[1], item[0],nodes)

    node_key_value = {}
    # node_domains = {}
    node_domains = [1,2,3,4,5,6,7,8,9]
    for index,node_value in enumerate(nodes):
      node_key_value[node_value] = index
      # node_domains[node_value] = [1,2,3,4,5,6,7,8,9]
      
      
    node_values = min_conflicts(node_graph, node_domains, max_steps, node_key_value)
    return node_values

### Test Cases

Below we've included 4 test cases to help you debug your code. Your submitted code will be tested on other cases as well, but if your implementation of the above Min-Conflicts search algorithm is able to solve these problems, you should be good to go.

In [13]:
# test Case 1

nodes = 'YGVRB'
arcs = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        break
        
all_solutions = [[1, 1, 1, 7, 2],[2, 1, 2, 4, 3],[2, 6, 7, 6, 1],[2, 8, 9, 6, 1],
                 [3, 3, 1, 5, 4],[6, 2, 8, 7, 1],[6, 7, 8, 2, 1],[6, 9, 4, 8, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

ERROR: False solution found: [4, 2, 7, 1, 9]


In [60]:
# test Case 2

nodes = 'YVBGR'
arcs = [(0,1), (0,2), (1,3), (2,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[1, 1, 1, 1, 9], [1, 3, 7, 3, 6], [1, 7, 3, 7, 2], [1, 9, 9, 9, 8]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YVBGR
ERROR: False solution found: [0, 0, 1, 0, 1]


In [49]:
# test Case 3

nodes = 'VYGBR'
arcs = [(0,1), (1,2), (2,3), (3,4)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[2, 2, 1, 9, 8],[3, 3, 1, 8, 7],[4, 4, 1, 7, 6],[5, 5, 1, 6, 5],[5, 5, 3, 8, 5],
                 [6, 6, 1, 5, 4],[7, 7, 1, 4, 3],[8, 8, 1, 3, 2],[8, 8, 6, 8, 2],[9, 9, 1, 2, 1]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

VYGBR
Solution found: [9, 9, 1, 2, 1]


In [45]:
# test Case 4

nodes = 'YGVBR'
arcs = [(0,1), (0,2), (1,3), (2,3), (3,4), (1,2)]
max_steps = 1000

for _ in range(max_steps):
    sol = solve_csp(nodes, arcs, max_steps)
    if sol != []:
        print(nodes)
        break
        
all_solutions = [[4, 4, 1, 9, 4],[4, 7, 2, 1, 1],[4, 7, 2, 1, 2],[4, 7, 2, 1, 3],[4, 7, 2, 1, 4],
                 [4, 7, 2, 1, 5],[4, 7, 2, 1, 6],[4, 7, 2, 1, 7],[4, 7, 2, 1, 8],[4, 7, 2, 1, 9],
                 [4, 8, 3, 1, 1],[4, 8, 3, 1, 2],[4, 8, 3, 1, 3],[4, 8, 3, 1, 4],[4, 8, 3, 1, 5],
                 [4, 8, 3, 1, 6],[4, 8, 3, 1, 7],[4, 8, 3, 1, 8],[5, 1, 5, 1, 4],[5, 1, 5, 1, 5],
                 [5, 1, 5, 1, 6],[5, 1, 5, 1, 7],[5, 1, 5, 1, 8],[5, 1, 5, 1, 9]]

if sol == []:
    print('No solution')
else:
    if sol in all_solutions:
        print('Solution found:', sol)
    else:
        print('ERROR: False solution found:', sol)

YGVBR
Solution found: [4, 7, 2, 1, 1]


## Problem 3: The N-Rooks Problem
---

Rooks can move any number of squares horizontally or vertically on a chess board. The n rooks problem is to arrange rooks on an $n\times n$ board in such a way that none of the rooks could bump into another by making any of its possible horizontal or vertical moves. 

For this problem, the variables are each column (labeled 0, 1, ... , $ n-1$), the the domain consists of each possible row (also labeled 0, 1, ... , $ n-1$). In each column we place a rook on row 0, row 1, ... , row $n-1$.

For example, if $n=2$ we have only two solutions to this problem:

    R @
    @ R 
    
or

    @ R
    R @ 
   

### Part 1)
How many possible solutions are there to this CSP, in terms of n?  Also, give a simple proof that your answer is correct.

There are n! solutions possible to the n-rook problem. The simplest solution is placing the first rook in cell[1][1] and the corresponding ith rook at cell[i][i]. Thus, ith rook can be placed in (n-i) positions in the row, which results in total permutations as n*(n-1)*(n-2)..2*1 which is n!


### Python Library to solve CSPs

One useful Python module for solving these types of problems is called *constraint*. In the next cell you'll see how to load this module into a Jupyter notebook running in Colab. 

We'll use this module to solve the following simple CSP. Suppose our variables are x and y, and the values they are allowed to assume are numbers in the domain {1, 2, ... , 100}, and with constraints be that $x^2=y$ and that $x$ is odd.

In [17]:
# The following line imports the constraint module into a Jupyter notebook running in Colab.
# (This only needs to be done once per session)

!pip install python-constraint

# Let's load all the functions available in this module
# (This only needs to be done once per session)

from constraint import *

# Now we initialize a new problem and add the two variables.
problem = Problem()
problem.addVariables(["x", "y"], list(range(1, 101)))

# So far there are no constraints. If we solve the problem with no constraints, 
# then every possible combination of x and y values from 1 to 100 are valid solutions.
solutions = problem.getSolutions()

# Total number of solutions
print( len(solutions) )

10000


In [18]:
# Add the given constraints:
problem.addConstraint(lambda a, b: a**2 == b and (a % 2) == 1, ("x", "y"))

# There are only 5 solutions that satisfy those constraints:
solutions = problem.getSolutions()
print( len(solutions) )

5


In [19]:
# Here are all 5 solutions:
print(solutions)

[{'x': 9, 'y': 81}, {'x': 7, 'y': 49}, {'x': 5, 'y': 25}, {'x': 3, 'y': 9}, {'x': 1, 'y': 1}]


Notice that the solutions consist of a list of dictionaries, where each dictionary represents a solution. For example, the first solution is {'x': 9, 'y': 81}, since with $x=9$ and $y=81$ it's true that $x^2=y$.

### Part 2) The n-rooks problem revisited

Modify the example before to solve the n-rooks problem.

In [20]:
### YOUR CODE HERE ###

n_rook_problem = Problem()
num_of_rooks = 4      # update the number of rooks as per the requirement
cols = range(num_of_rooks)
rows = range(num_of_rooks)
n_rook_problem.addVariables(cols, rows)
for col1 in cols:
  for col2 in cols:
    if col1 < col2:
      n_rook_problem.addConstraint(lambda row1, row2: row1 != row2,(col1, col2))
solutions = n_rook_problem.getSolutions()
print(solutions)
print(len(solutions))

[{0: 3, 1: 2, 2: 1, 3: 0}, {0: 3, 1: 2, 2: 0, 3: 1}, {0: 3, 1: 1, 2: 2, 3: 0}, {0: 3, 1: 1, 2: 0, 3: 2}, {0: 3, 1: 0, 2: 1, 3: 2}, {0: 3, 1: 0, 2: 2, 3: 1}, {0: 2, 1: 3, 2: 0, 3: 1}, {0: 2, 1: 3, 2: 1, 3: 0}, {0: 2, 1: 1, 2: 3, 3: 0}, {0: 2, 1: 1, 2: 0, 3: 3}, {0: 2, 1: 0, 2: 1, 3: 3}, {0: 2, 1: 0, 2: 3, 3: 1}, {0: 1, 1: 2, 2: 0, 3: 3}, {0: 1, 1: 2, 2: 3, 3: 0}, {0: 1, 1: 3, 2: 2, 3: 0}, {0: 1, 1: 3, 2: 0, 3: 2}, {0: 1, 1: 0, 2: 3, 3: 2}, {0: 1, 1: 0, 2: 2, 3: 3}, {0: 0, 1: 1, 2: 3, 3: 2}, {0: 0, 1: 1, 2: 2, 3: 3}, {0: 0, 1: 2, 2: 1, 3: 3}, {0: 0, 1: 2, 2: 3, 3: 1}, {0: 0, 1: 3, 2: 2, 3: 1}, {0: 0, 1: 3, 2: 1, 3: 2}]
24


### Part 3)

*   How many ways are there of arranging $8$ rooks on an $8\times8$ board so that none impede the others?
*   How many ways are there arranging $11$ rooks on an $11\times11$ board so that none impede the others?


1. 8! = 40,320
2. 11! = 39,916,800
